# 🎯 Objektu Detekzioa - Sarrera

**Egilea**: Mikel Aldalur Corta  
**Data**: 2025  
**Maila**: Aurreratua  

---

## 🎯 Helburua

Notebook honetan **Objektu Detekzioa** kontzeptua ikasiko dugu:
- Zer da Objektu Detekzioa?
- Klasifikazio vs Detekzioa vs Segmentazioa
- Algoritmo nagusiak (YOLO, R-CNN, SSD)
- Praktika sinpleak

---

## 📚 Ikasiko duguna

1. ✅ Objektu Detekzioaren oinarriak
2. ✅ Bounding Box-ak
3. ✅ IoU (Intersection over Union)
4. ✅ Non-Maximum Suppression (NMS)
5. ✅ Algoritmo nagusiak

## 1️⃣ Zer da Objektu Detekzioa?

**Objektu Detekzioa** irudi batean objektuak aurkitzea eta lokalizatzea da.

### 🔑 Zer egiten du?

1. **Klasifikazioa**: Zer objektu da?
2. **Lokalizazioa**: Non dago objektua?

### 📝 Aplikazioak

- 🚗 **Autonomous Driving**: Ibilgailuak, oinezko pertsonak, seinaleak detektatu
- 🏥 **Medikuntza**: Tumoreak, anomaliak detektatu
- 🔒 **Segurtasuna**: Pertsonak, objektu susmagarriak detektatu
- 🛒 **Retail**: Produktuak, erosketa jokabideak analizatu

---

### 📊 Klasifikazio vs Detekzioa vs Segmentazioa

| Ataza | Deskribapena | Irteerak |
|-------|--------------|----------|
| **Klasifikazioa** | Irudia klasifikatu | Klase bat |
| **Detekzioa** | Objektuak detektatu + lokalizatu | Bounding boxes + klaseak |
| **Segmentazioa** | Pixel bakoitza klasifikatu | Segmentation mask |

## 2️⃣ Liburutegiak Inportatu

In [ ]:
# Liburutegiak
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from PIL import Image, ImageDraw, ImageFont
import warnings
warnings.filterwarnings('ignore')

# Estiloa
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
np.random.seed(42)

print("✅ Liburutegiak kargatuta!")

## 3️⃣ Bounding Box Kontzeptua

In [ ]:
# Bounding Box funtzioak
def marraztu_bounding_box(ax, bbox, label=None, color='red', linewidth=2):
    """
    Bounding box bat marraztu
    bbox: (x, y, width, height)
    """
    x, y, w, h = bbox
    rect = patches.Rectangle(
        (x, y), w, h,
        linewidth=linewidth,
        edgecolor=color,
        facecolor='none'
    )
    ax.add_patch(rect)
    
    if label:
        ax.text(x, y-5, label, 
               bbox=dict(facecolor=color, alpha=0.7),
               fontsize=10, color='white', fontweight='bold')

# Adibidea
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

# Irudi simulatua
img = np.random.rand(400, 400, 3)
ax.imshow(img)

# Bounding box-ak marraztu
bboxes = [
    ((50, 50, 100, 150), 'Persona', 'red'),
    ((200, 100, 120, 100), 'Kotxea', 'blue'),
    ((100, 250, 80, 80), 'Txakurra', 'green')
]

for bbox, label, color in bboxes:
    marraztu_bounding_box(ax, bbox, label, color)

ax.set_title('🎯 Bounding Box Adibidea', fontsize=16, fontweight='bold', pad=20)
ax.axis('off')
plt.tight_layout()
plt.show()

print("✅ Bounding box-ak marraztuta!")

## 4️⃣ IoU (Intersection over Union)

**IoU** bounding box biren arteko **overlap-a** neurtzen duen metrika bat da.

$$
IoU = \frac{Area\ of\ Overlap}{Area\ of\ Union}
$$

- IoU = 0: Ez dago overlap-ik
- IoU = 1: Overlap osoa
- IoU > 0.5: Onargarria
- IoU > 0.7: Ona

In [ ]:
# IoU funtzioa
def kalkulatu_iou(box1, box2):
    """
    IoU kalkulatu bi bounding box-en artean
    box: (x, y, width, height)
    """
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    
    # Overlap area kalkulatu
    x_overlap = max(0, min(x1 + w1, x2 + w2) - max(x1, x2))
    y_overlap = max(0, min(y1 + h1, y2 + h2) - max(y1, y2))
    overlap_area = x_overlap * y_overlap
    
    # Union area kalkulatu
    area1 = w1 * h1
    area2 = w2 * h2
    union_area = area1 + area2 - overlap_area
    
    # IoU kalkulatu
    iou = overlap_area / union_area if union_area > 0 else 0
    
    return iou

# Adibideak
box1 = (50, 50, 100, 100)
box2 = (75, 75, 100, 100)
box3 = (200, 200, 100, 100)

iou_12 = kalkulatu_iou(box1, box2)
iou_13 = kalkulatu_iou(box1, box3)

print(f"📊 IoU adibideak:")
print(f"  Box1 vs Box2 (overlap handia): {iou_12:.3f}")
print(f"  Box1 vs Box3 (overlap-ik ez): {iou_13:.3f}")

# Bistaratu
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Overlap handia
axes[0].set_xlim(0, 300)
axes[0].set_ylim(0, 300)
axes[0].invert_yaxis()
marraztu_bounding_box(axes[0], box1, 'Box1', 'red')
marraztu_bounding_box(axes[0], box2, 'Box2', 'blue')
axes[0].set_title(f'🎯 Overlap Handia (IoU={iou_12:.3f})', 
                  fontsize=14, fontweight='bold')
axes[0].axis('off')

# Overlap-ik ez
axes[1].set_xlim(0, 350)
axes[1].set_ylim(0, 350)
axes[1].invert_yaxis()
marraztu_bounding_box(axes[1], box1, 'Box1', 'red')
marraztu_bounding_box(axes[1], box3, 'Box3', 'green')
axes[1].set_title(f'🎯 Overlap-ik Ez (IoU={iou_13:.3f})', 
                  fontsize=14, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("\n✅ IoU adibideak bistaratuta!")

## 5️⃣ Non-Maximum Suppression (NMS)

**NMS** detekzio anizkoitzak kentzeko erabiltzen den teknika bat da.

### 🔑 Nola funtzionatzen du?

1. Detekzioak confidence-z ordenatu
2. Confidence altuena aukeratu
3. IoU > threshold dituzten detekzioak kendu
4. Errepikatu

NMS-ek overlap handiko detekzioak kentzen ditu eta bakarrik onena uzten du.

In [ ]:
# NMS funtzioa
def non_maximum_suppression(boxes, confidences, iou_threshold=0.5):
    """
    Non-Maximum Suppression aplikatu
    boxes: lista [(x, y, w, h), ...]
    confidences: lista [confidence1, confidence2, ...]
    """
    # Hutsa bada, bueltatu
    if len(boxes) == 0:
        return []
    
    # Confidence-z ordenatu (handienetik txikienera)
    indices = np.argsort(confidences)[::-1]
    
    # Gordeko ditugun indizeak
    keep_indices = []
    
    while len(indices) > 0:
        # Lehenengo indizea gorde
        current = indices[0]
        keep_indices.append(current)
        
        # Gainerako box-ekin IoU kalkulatu
        ious = []
        for idx in indices[1:]:
            iou = kalkulatu_iou(boxes[current], boxes[idx])
            ious.append(iou)
        
        # IoU < threshold dituztenak gorde
        ious = np.array(ious)
        indices = indices[1:][ious < iou_threshold]
    
    return keep_indices

# Adibidea
boxes = [
    (50, 50, 100, 100),
    (60, 60, 100, 100),
    (55, 55, 100, 100),
    (200, 200, 80, 80)
]
confidences = [0.9, 0.85, 0.88, 0.95]

keep_indices = non_maximum_suppression(boxes, confidences, iou_threshold=0.5)

print(f"📊 NMS adibidea:")
print(f"  Hasierako detekzioak: {len(boxes)}")
print(f"  NMS ondorengo detekzioak: {len(keep_indices)}")
print(f"  Gordetako indizeak: {keep_indices}")

# Bistaratu
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# NMS aurretik
axes[0].set_xlim(0, 350)
axes[0].set_ylim(0, 350)
axes[0].invert_yaxis()
for i, (box, conf) in enumerate(zip(boxes, confidences)):
    marraztu_bounding_box(axes[0], box, f'Det {i+1} ({conf:.2f})', 
                         ['red', 'blue', 'green', 'orange'][i])
axes[0].set_title('🎯 NMS Aurretik', fontsize=14, fontweight='bold')
axes[0].axis('off')

# NMS ondoren
axes[1].set_xlim(0, 350)
axes[1].set_ylim(0, 350)
axes[1].invert_yaxis()
for idx in keep_indices:
    marraztu_bounding_box(axes[1], boxes[idx], 
                         f'Det {idx+1} ({confidences[idx]:.2f})', 
                         ['red', 'blue', 'green', 'orange'][idx])
axes[1].set_title('🎯 NMS Ondoren', fontsize=14, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("\n✅ NMS adibidea bistaratuta!")

## 6️⃣ Algoritmo Nagusiak

### 1. **R-CNN familia**

- **R-CNN**: Region-based CNN
- **Fast R-CNN**: R-CNN-ren bertsioa azkarragoa
- **Faster R-CNN**: Region Proposal Network (RPN) erabiltzen du
- **Mask R-CNN**: Segmentazioa ere egiten du

### 2. **YOLO (You Only Look Once)**

- Irudi osoa behin bakarrik prozesatzen du
- Oso azkarra (real-time)
- Zehaztasuna pixka bat txikiagoa

### 3. **SSD (Single Shot Detector)**

- YOLO-ren antzekoa
- Multi-scale feature maps erabiltzen ditu
- Zehaztasun ona eta abiadura ona

---

### 📊 Konparaketa

| Algoritmoa | Abiadura | Zehaztasuna | Erabilera |
|-----------|----------|-------------|-----------|
| **Faster R-CNN** | Motela | Handia | Zehaztasun kritikoa |
| **YOLO** | Oso azkarra | Ona | Real-time aplikazioak |
| **SSD** | Azkarra | Ona | Oreka ona |
| **Mask R-CNN** | Motela | Handia | Segmentazioa behar |

## 7️⃣ YOLO Arkitektura (Kontzeptua)

YOLO-k irudi bat **grid-etan** banatzen du eta grid bakoitzean objektuak detektatzen ditu.

### 🔑 Nola funtzionatzen du?

1. **Grid-etan banatu**: Adibidez, 7x7 grid
2. **Bounding box-ak aurreikusi**: Grid bakoitzak B bounding box aurreikusten ditu
3. **Confidence score**: Bakoitzak confidence score bat du
4. **Klasea aurreikusi**: Grid bakoitzak klase bat aurreikusten du
5. **NMS aplikatu**: Detekzio anizkoitzak kentzeko


In [ ]:
# YOLO grid simulazioa
def marraztu_yolo_grid(grid_size=7):
    """
    YOLO grid simulazioa marraztu
    """
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    
    # Irudi simulatua
    img = np.random.rand(448, 448, 3) * 0.5
    ax.imshow(img)
    
    # Grid marraztu
    step = 448 // grid_size
    for i in range(grid_size + 1):
        ax.axhline(y=i*step, color='white', linewidth=1, alpha=0.5)
        ax.axvline(x=i*step, color='white', linewidth=1, alpha=0.5)
    
    # Objektu batzuk simulatu
    objects = [
        ((100, 100, 80, 120), 'Persona', 'red', 0.95),
        ((250, 200, 100, 80), 'Kotxea', 'blue', 0.88),
        ((350, 50, 60, 60), 'Txakurra', 'green', 0.92)
    ]
    
    for bbox, label, color, conf in objects:
        marraztu_bounding_box(ax, bbox, f'{label} ({conf:.2f})', color)
    
    ax.set_title(f'🎯 YOLO Grid ({grid_size}x{grid_size})', 
                fontsize=16, fontweight='bold', pad=20)
    ax.axis('off')
    plt.tight_layout()
    plt.show()

# Grid-a marraztu
marraztu_yolo_grid(grid_size=7)

print("✅ YOLO grid simulazioa bistaratuta!")

## 8️⃣ Metrikak - mAP (mean Average Precision)

**mAP** objektu detekzioaren metrika estandarra da.

### 📊 Nola kalkulatzen da?

1. **Precision eta Recall kalkulatu** IoU threshold-arekin
2. **Precision-Recall kurba** marraztu
3. **Average Precision (AP)** kalkulatu klase bakoitzerako
4. **mAP**: Klase guztien AP-ren batezbestekoa

---

### 🔑 Threshold-ak

- **IoU > 0.5**: PASCAL VOC metrika
- **IoU > 0.5:0.95**: COCO metrika (zorrotzagoa)

In [ ]:
# Precision-Recall kurba simulatua
def marraztu_precision_recall_kurba():
    """
    Precision-Recall kurba simulatua marraztu
    """
    # Simulazioa
    recall = np.linspace(0, 1, 100)
    precision = 1 / (1 + np.exp(5 * (recall - 0.5))) + np.random.rand(100) * 0.1
    precision = np.clip(precision, 0, 1)
    
    # AP kalkulatu (integral)
    ap = np.trapz(precision, recall)
    
    # Marraztu
    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    ax.plot(recall, precision, linewidth=3, color='blue')
    ax.fill_between(recall, precision, alpha=0.3)
    ax.set_xlabel('Recall', fontsize=14)
    ax.set_ylabel('Precision', fontsize=14)
    ax.set_title(f'📊 Precision-Recall Kurba (AP={ap:.3f})', 
                fontsize=16, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1.05])
    
    plt.tight_layout()
    plt.show()
    
    return ap

ap = marraztu_precision_recall_kurba()
print(f"✅ Average Precision (AP): {ap:.3f}")

## 📝 Ondorioak

### 🎯 Ikasitako Kontzeptuak

- **Objektu Detekzioa**: Klasifikazioa + Lokalizazioa
- **Bounding Box**: Objektuak lokalizatzeko
- **IoU**: Overlap-a neurtzeko metrika
- **NMS**: Detekzio anizkoitzak kentzeko
- **Algoritmoak**: R-CNN, YOLO, SSD

### 📊 Algoritmo Konparaketa

| Algoritmoa | Abantailak | Desabantailak |
|-----------|-----------|---------------|
| **R-CNN** | Zehaztasun handia | Oso motela |
| **Fast R-CNN** | Azkarragoa | Oraindik motela |
| **Faster R-CNN** | Zehaztasun handia | Abiadura ertaina |
| **YOLO** | Oso azkarra (real-time) | Zehaztasun txikiagoa |
| **SSD** | Oreka ona | - |
| **Mask R-CNN** | Segmentazioa ere | Oso motela |

### 🚀 Hurrengo Pausoak

- **YOLO praktika**: YOLOv8 erabili detekziorako
- **Kasu errealak**: Bideo batean objektuak detektatu
- **Custom dataset**: Gure objektuak detektatzeko eredua entrenatu
- **Aurpegi Ezagutza**: Face detection eta recognition

---

**Egilea**: Mikel Aldalur Corta  
**Data**: 2025  
**Lizentzia**: MIT